In [250]:
import pandas as pd
import numpy as np
import seaborn as sns                       #visualisation
import matplotlib.pyplot as plt             #visualisation
%matplotlib inline     
sns.set(color_codes=True)

#modeling

from xgboost import XGBRegressor 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, RidgeCV
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score, train_test_split, KFold, StratifiedGroupKFold, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn import ensemble
from sklearn import neighbors
from sklearn.metrics import mean_squared_error 
from math import sqrt
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

In [251]:
# LSTM modeling
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
from pandas import DataFrame , concat
from sklearn.metrics import mean_absolute_error , mean_squared_error
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from numpy import mean , concatenate
from math import sqrt
from pandas import read_csv
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Activation
from numpy import array , hstack
from tensorflow import keras
import tensorflow as tf
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()
# if you're using google drive to store the dataset
#from google.colab import drive

In [252]:
data_path = '/global/scratch/users/yaqiantang/discoveryData/'

# 5th Site - US-Var

## Data Pre-processing

In [253]:
site_id = 'US-Var' # 
df = pd.read_csv(data_path + 'data_full_half_hourly_raw_v0_1_' + site_id + '.csv')
df.head()

,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,...,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen
0,200011010000,200011010030,8.59,0,8.117,0.0,0.0,0,0.0,297.554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GRA,Temperate
1,200011010030,200011010100,8.54,0,7.898,0.0,0.0,0,0.0,297.554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GRA,Temperate
2,200011010100,200011010130,8.51,0,7.680,0.0,0.0,0,0.0,277.808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GRA,Temperate
3,200011010130,200011010200,8.07,0,7.353,0.0,0.0,0,0.0,277.808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GRA,Temperate
4,200011010200,200011010230,7.43,0,7.027,0.0,0.0,0,0.0,277.808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GRA,Temperate


In [254]:
df.columns

Index(['TIMESTAMP_START', 'TIMESTAMP_END', 'TA_F', 'TA_F_QC', 'TA_ERA',
       'SW_IN_POT', 'SW_IN_F', 'SW_IN_F_QC', 'SW_IN_ERA', 'LW_IN_F',
       'LW_IN_F_QC', 'LW_IN_ERA', 'VPD_F', 'VPD_F_QC', 'VPD_ERA', 'P_F',
       'P_F_QC', 'P_ERA', 'PA_F', 'PA_F_QC', 'PA_ERA', 'NETRAD', 'PPFD_IN',
       'G_F_MDS', 'G_F_MDS_QC', 'LE_F_MDS', 'LE_F_MDS_QC', 'LE_CORR',
       'H_F_MDS', 'H_F_MDS_QC', 'H_CORR', 'NEE_VUT_REF', 'NEE_VUT_REF_QC',
       'NEE_CUT_REF', 'NEE_CUT_REF_QC', 'GPP_NT_VUT_REF', 'GPP_DT_VUT_REF',
       'GPP_NT_CUT_REF', 'GPP_DT_CUT_REF', 'RECO_NT_VUT_REF',
       'RECO_DT_VUT_REF', 'RECO_NT_CUT_REF', 'RECO_DT_CUT_REF', 'datetime',
       'year', 'month', 'day', 'hour', 'SITE_ID', 'date', 'NEE_VUT_REF_qa',
       'SW_DIF', 'EVI', 'NDVI', 'NIRv', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
       'b7', 'IGBP', 'koppen'],
      dtype='object')

In [255]:
def check_half_hour(time_1, time_2, pattern):
    if (time_1 + pattern) % 10000 // 100 == 24:
        time_1 = ((time_1 // 10000) + 1 ) * 10000
        if time_1 == time_2:
            return True
        else:
            return False
    else:
        if time_1 + pattern != time_2:
            return False
        else:
            return True

def clean_data_gap(df, pattern=[30, 70]):
    df_start_time = df.loc[:, "TIMESTAMP_START"]
    pattern_index = 0
    df_index = 0
    end = len(df_start_time) - 2

    while df_index != end:
        if check_half_hour(df_start_time[df_index], df_start_time[df_index+1], pattern[pattern_index]) == False and (df_start_time[df_index+1] - df_start_time[df_index]) <= 30000:
            print(df_start_time[df_index+1])
            print(df_start_time[df_index])
            print(pattern[pattern_index])
            curr = df.loc[df_index]
            
            if (curr["TIMESTAMP_START"] + pattern[pattern_index]) % 10000 // 100 == 24:
                curr["TIMESTAMP_START"] = ((curr["TIMESTAMP_START"] // 10000) + 1) * 10000
            else: 
                curr["TIMESTAMP_START"] += pattern[pattern_index]
            
            df.loc[df_index+0.5] = curr
            df = df.sort_index().reset_index(drop=True)
            df_start_time = df.loc[:, "TIMESTAMP_START"]

            df_index += 1
            pattern_index += 1
            if pattern_index == len(pattern):
                pattern_index = 0
        else:
            df_index += 1
            pattern_index += 1
            if pattern_index == len(pattern):
                pattern_index = 0
    return df

In [256]:
df = clean_data_gap(df)

In [257]:
df.head()

,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,...,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen
0,200011010000,200011010030,8.59,0,8.117,0.0,0.0,0,0.0,297.554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GRA,Temperate
1,200011010030,200011010100,8.54,0,7.898,0.0,0.0,0,0.0,297.554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GRA,Temperate
2,200011010100,200011010130,8.51,0,7.680,0.0,0.0,0,0.0,277.808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GRA,Temperate
3,200011010130,200011010200,8.07,0,7.353,0.0,0.0,0,0.0,277.808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GRA,Temperate
4,200011010200,200011010230,7.43,0,7.027,0.0,0.0,0,0.0,277.808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GRA,Temperate


In [258]:
target = 'GPP_NT_VUT_REF'
features = ['year','TA_ERA','SW_IN_ERA','P_ERA','VPD_ERA','b1','b2','b3','b4','b5','b6',
            'b7','NDVI','EVI','NIRv']
# and categorical features: IGBP and koppen

df['datetime'] = pd.to_datetime(df['datetime'])
df = df.set_index('datetime')
df.head()

,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,...,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen
datetime,,,,,,,,,,,,,,,,,,,,,
2000-11-01 00:00:00,200011010000,200011010030,8.59,0,8.117,0.0,0.0,0,0.0,297.554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GRA,Temperate
2000-11-01 00:30:00,200011010030,200011010100,8.54,0,7.898,0.0,0.0,0,0.0,297.554,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GRA,Temperate
2000-11-01 01:00:00,200011010100,200011010130,8.51,0,7.680,0.0,0.0,0,0.0,277.808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GRA,Temperate
2000-11-01 01:30:00,200011010130,200011010200,8.07,0,7.353,0.0,0.0,0,0.0,277.808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GRA,Temperate
2000-11-01 02:00:00,200011010200,200011010230,7.43,0,7.027,0.0,0.0,0,0.0,277.808,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GRA,Temperate


In [259]:
# One hot encoding categorical variables: IGBP and Koppen
df_ohe = df[[target, 'IGBP', 'koppen'] + features]

categorical_columns = ['IGBP', 'koppen']
for col in categorical_columns:
    col_ohe = pd.get_dummies(df[col], prefix=col)
    df_ohe = pd.concat((df_ohe, col_ohe), axis=1).drop(col, axis=1)

df_ohe.columns

Index(['GPP_NT_VUT_REF', 'year', 'TA_ERA', 'SW_IN_ERA', 'P_ERA', 'VPD_ERA',
       'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'NDVI', 'EVI', 'NIRv',
       'IGBP_GRA', 'koppen_Temperate'],
      dtype='object')

In [260]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    return df

In [261]:
# clean the data.
df = clean_dataset(df_ohe)
df.head()

,GPP_NT_VUT_REF,year,TA_ERA,SW_IN_ERA,P_ERA,VPD_ERA,b1,b2,b3,b4,b5,b6,b7,NDVI,EVI,NIRv,IGBP_GRA,koppen_Temperate
datetime,,,,,,,,,,,,,,,,,,
2001-01-01 00:00:00,-1.84180,2001,2.171,0.0,0.0,0.893,0.0628,0.2694,0.0304,0.0625,0.3155,0.2595,0.1311,0.621915,0.364194,0.167544,1,1
2001-01-01 00:30:00,-1.91023,2001,2.161,0.0,0.0,1.107,0.0628,0.2694,0.0304,0.0625,0.3155,0.2595,0.1311,0.621915,0.364194,0.167544,1,1
2001-01-01 01:00:00,-1.85277,2001,2.150,0.0,0.0,1.321,0.0628,0.2694,0.0304,0.0625,0.3155,0.2595,0.1311,0.621915,0.364194,0.167544,1,1
2001-01-01 01:30:00,-5.84140,2001,2.044,0.0,0.0,1.391,0.0628,0.2694,0.0304,0.0625,0.3155,0.2595,0.1311,0.621915,0.364194,0.167544,1,1
2001-01-01 02:00:00,-2.51478,2001,1.938,0.0,0.0,1.461,0.0628,0.2694,0.0304,0.0625,0.3155,0.2595,0.1311,0.621915,0.364194,0.167544,1,1


In [262]:
# test train split grouped by SiteID, year, month
from sklearn.model_selection import GroupShuffleSplit 

splitter = GroupShuffleSplit(test_size=0.3, n_splits=2, random_state=42)
split = splitter.split(df, groups=df['year'])
train_inds, test_inds = next(split)

train_df = df.iloc[train_inds]
test_df = df.iloc[test_inds]

In [263]:
train_df.head(2)

,GPP_NT_VUT_REF,year,TA_ERA,SW_IN_ERA,P_ERA,VPD_ERA,b1,b2,b3,b4,b5,b6,b7,NDVI,EVI,NIRv,IGBP_GRA,koppen_Temperate
datetime,,,,,,,,,,,,,,,,,,
2002-01-09 00:00:00,-0.556452,2002,7.249,0.0,0.0,0.976,0.0641,0.263,0.0315,0.0688,0.2799,0.2591,0.1309,0.608071,0.352322,0.159923,1,1
2002-01-09 00:30:00,-0.562734,2002,6.956,0.0,0.0,0.933,0.0641,0.263,0.0315,0.0688,0.2799,0.2591,0.1309,0.608071,0.352322,0.159923,1,1


In [264]:
test_df.head(2)

,GPP_NT_VUT_REF,year,TA_ERA,SW_IN_ERA,P_ERA,VPD_ERA,b1,b2,b3,b4,b5,b6,b7,NDVI,EVI,NIRv,IGBP_GRA,koppen_Temperate
datetime,,,,,,,,,,,,,,,,,,
2001-01-01 00:00:00,-1.84180,2001,2.171,0.0,0.0,0.893,0.0628,0.2694,0.0304,0.0625,0.3155,0.2595,0.1311,0.621915,0.364194,0.167544,1,1
2001-01-01 00:30:00,-1.91023,2001,2.161,0.0,0.0,1.107,0.0628,0.2694,0.0304,0.0625,0.3155,0.2595,0.1311,0.621915,0.364194,0.167544,1,1


In [265]:
train_df['year'].value_counts()

2007    17520
2009    17520
2011    17472
2008    17184
2003    17136
2002    17040
2004    16992
2014    16896
2005    15216
Name: year, dtype: int64

In [266]:
test_df['year'].value_counts()

2012    17568
2013    17520
2006    17376
2001    17136
2010    16176
Name: year, dtype: int64

In [267]:
X_train = train_df.drop([target, 'year'], axis = 1)
X_train.head(2)

,TA_ERA,SW_IN_ERA,P_ERA,VPD_ERA,b1,b2,b3,b4,b5,b6,b7,NDVI,EVI,NIRv,IGBP_GRA,koppen_Temperate
datetime,,,,,,,,,,,,,,,,
2002-01-09 00:00:00,7.249,0.0,0.0,0.976,0.0641,0.263,0.0315,0.0688,0.2799,0.2591,0.1309,0.608071,0.352322,0.159923,1,1
2002-01-09 00:30:00,6.956,0.0,0.0,0.933,0.0641,0.263,0.0315,0.0688,0.2799,0.2591,0.1309,0.608071,0.352322,0.159923,1,1


In [268]:
y_train = train_df[target]
y_train.head(2)

datetime
2002-01-09 00:00:00   -0.556452
2002-01-09 00:30:00   -0.562734
Name: GPP_NT_VUT_REF, dtype: float64

In [269]:
X_test = test_df.drop([target, 'year'], axis = 1)
X_test.head(2)

,TA_ERA,SW_IN_ERA,P_ERA,VPD_ERA,b1,b2,b3,b4,b5,b6,b7,NDVI,EVI,NIRv,IGBP_GRA,koppen_Temperate
datetime,,,,,,,,,,,,,,,,
2001-01-01 00:00:00,2.171,0.0,0.0,0.893,0.0628,0.2694,0.0304,0.0625,0.3155,0.2595,0.1311,0.621915,0.364194,0.167544,1,1
2001-01-01 00:30:00,2.161,0.0,0.0,1.107,0.0628,0.2694,0.0304,0.0625,0.3155,0.2595,0.1311,0.621915,0.364194,0.167544,1,1


In [270]:
y_test = test_df[target]
y_test.head(2)

datetime
2001-01-01 00:00:00   -1.84180
2001-01-01 00:30:00   -1.91023
Name: GPP_NT_VUT_REF, dtype: float64

In [271]:
train_df.shape

(152976, 18)

In [272]:
test_df.shape

(85776, 18)

In [273]:
train_df = train_df[['TA_ERA','SW_IN_ERA','P_ERA','VPD_ERA','b1','b2','b3','b4','b5','b6',
            'b7','NDVI','EVI','NIRv', target]]

In [274]:
test_df = test_df[['TA_ERA','SW_IN_ERA','P_ERA','VPD_ERA','b1','b2','b3','b4','b5','b6',
            'b7','NDVI','EVI','NIRv', target]]

In [275]:
scaler = MinMaxScaler(feature_range=(0, 1))
data = np.vstack((train_df, test_df))
data_scaled = scaler.fit_transform(data)
train_scaled = data_scaled[:152976,]
test_scaled = data_scaled[152976:,]
train_scaled.shape, test_scaled.shape

((152976, 15), (85776, 15))

In [276]:
len(test_scaled)

85776

In [290]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
  # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
        #i += n_steps_in
    return array(X), array(y)


# choose a number of time steps #change this accordingly
n_steps_in, n_steps_out = 12, 12 #144,144
# covert into input/output
train_X, train_y = split_sequences(train_scaled, n_steps_in, n_steps_out)
test_X, test_y = split_sequences(test_scaled, n_steps_in, n_steps_out)

print ("train_X.shape" , train_X.shape) 
print ("train_y.shape" , train_y.shape)
print ("test_X.shape" , test_X.shape) 
print ("test_y.shape" , test_y.shape)

train_X.shape (152954, 12, 14)
train_y.shape (152954, 12)
test_X.shape (85754, 12, 14)
test_y.shape (85754, 12)


## Time Series LSTM

In [291]:
#optimizer learning rate
opt = keras.optimizers.Adam(learning_rate=0.0001)
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps_in, 14)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(n_steps_out))
model.add(Activation('linear'))
model.compile(loss='mse' , optimizer=opt , metrics=['mse'])

In [292]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_18 (LSTM)              (None, 12, 50)            13000     
                                                                 
 lstm_19 (LSTM)              (None, 50)                20200     
                                                                 
 dense_8 (Dense)             (None, 12)                612       
                                                                 
 activation_8 (Activation)   (None, 12)                0         
                                                                 
Total params: 33,812
Trainable params: 33,812
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_X, train_y, epochs=100, batch_size=1, verbose=1)

Epoch 1/100
 23098/152954 [===>..........................] - ETA: 4:18:12 - loss: 0.0030 - mse: 0.0030

In [ ]:
trainPredict = model.predict(train_X)
testPredict = model.predict(test_X)

In [ ]:
trainScore = np.sqrt(mean_squared_error(train_y, trainPredict))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = np.sqrt(mean_squared_error(test_y, testPredict))
print('Test Score: %.2f RMSE' % (testScore))

# 6th Site - ES-LJu

In [247]:
site_id = 'ES-LJu' # 
df1 = pd.read_csv(data_path + 'data_full_half_hourly_raw_v0_1_' + site_id + '.csv')
df1.head()

,TIMESTAMP_START,TIMESTAMP_END,TA_F,TA_F_QC,TA_ERA,SW_IN_POT,SW_IN_F,SW_IN_F_QC,SW_IN_ERA,LW_IN_F,...,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen
0,200405260000,200405260030,7.856,0,8.325,0.0,0.0,0,0.0,314.215,...,0.071307,0.1108,0.2183,0.052,0.0884,0.262,0.2695,0.1945,OSH,Temperate
1,200405260030,200405260100,7.650,0,8.180,0.0,0.0,0,0.0,314.215,...,0.071307,0.1108,0.2183,0.052,0.0884,0.262,0.2695,0.1945,OSH,Temperate
2,200405260100,200405260130,7.672,0,8.035,0.0,0.0,0,0.0,328.276,...,0.071307,0.1108,0.2183,0.052,0.0884,0.262,0.2695,0.1945,OSH,Temperate
3,200405260130,200405260200,7.793,0,8.300,0.0,0.0,0,0.0,328.276,...,0.071307,0.1108,0.2183,0.052,0.0884,0.262,0.2695,0.1945,OSH,Temperate
4,200405260200,200405260230,7.638,0,8.565,0.0,0.0,0,0.0,339.403,...,0.071307,0.1108,0.2183,0.052,0.0884,0.262,0.2695,0.1945,OSH,Temperate


In [249]:
df1 = clean_data_gap(df1)